In [1]:
!pip install --upgrade git+https://github.com/broadinstitute/keras-resnet

The directory '/home/ubuntu/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/ubuntu/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
  Cloning https://github.com/broadinstitute/keras-resnet to /tmp/pip-req-build-nfcc63tn
Requirement not upgraded as not directly required: keras in ./anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (from keras-resnet==0.1.0) (2.1.6)
Requirement not upgraded as not directly required: h5py in ./anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (from keras->keras-resnet==0.1.0) (2.7.1)
Requirement not upgraded as not directly required: scipy>=0.14 in ./anaconda3/envs/tensorflow_p36/lib/p

In [2]:
import warnings
import keras
import keras_resnet
import keras_resnet.models
import keras_resnet.blocks
import keras_resnet.layers
import keras.backend as K

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
import tensorflow as tf
import numpy as np
import sys
sys.path.insert(0, '/home/ubuntu/sbnet-new/sbnet/sbnet_tensorflow/benchmark')
import sparse_conv_lib

In [6]:
pwd

'/home/ubuntu'

In [7]:
sbnet_module = tf.load_op_library('/home/ubuntu/sbnet-new/sbnet/sbnet_tensorflow/sbnet_ops/build/libsbnet.so')

In [24]:
print(sbnet_module.__dict__.keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtins__', '_collections', '_op_def_pb2', '_common_shapes', '_op_def_registry', '_ops', '_op_def_library', 'tf_export', 'sparse_gather', 'sparse_scatter_var', 'sparse_scatter', 'cuda_timer_start', 'cuda_timer_end', '_reduce_mask_outputs', '_ReduceMaskOutput', 'reduce_mask', '_InitOpDefLibrary', '_op_def_lib', 'LIB_HANDLE', 'OP_LIST'])


In [25]:
b_size = 1

In [26]:
parameters = {
    "kernel_initializer": "he_normal",
}

In [27]:
def generate_mask(xsize, sparsity):
    """
    Generates a square top-left mask with a target sparsity value.
    :param xsize:       [list]      List of 4 int.
    :param sparsity:    [float]     Target sparsity value.
    :return:            [Tensor]    A tensor with shape to be `xsize` and contains a square of 1's
                                    and the rest being 0's.
    """
    density = 1.0 - sparsity
    edge_ratio = np.sqrt(density)
    height = tf.cast(tf.ceil(edge_ratio * xsize[1]), tf.int32)
    width = tf.cast(tf.ceil(edge_ratio * xsize[2]), tf.int32)
    x = tf.Variable(tf.convert_to_tensor(np.round(np.random.rand(1, 224, 224)), dtype=np.float32))
    #x[:, :height, :width] = 1.0
    return x

In [28]:
def bottleneck_2d(filters, stage=0, block=0, kernel_size=3, numerical_name=False, stride=1, freeze_bn=False):

    if keras.backend.image_data_format() == "channels_last":
        axis = 3
    else:
        axis = 1

    if block > 0 and numerical_name:
        block_char = "b{}".format(block)
    else:
        block_char = chr(ord('a') + block)

    stage_char = str(stage + 2)

    def f(x):
        y = keras.layers.Conv2D(filters, (1, 1), strides=stride, use_bias=False, name="res{}{}_branch2a".format(stage_char, block_char), **parameters)(x)
        y = keras_resnet.layers.BatchNormalization(axis=axis, epsilon=1e-5, freeze=freeze_bn, name="bn{}{}_branch2a".format(stage_char, block_char))(y)
        y = keras.layers.Activation("relu", name="res{}{}_branch2a_relu".format(stage_char, block_char))(y)

        y = keras.layers.ZeroPadding2D(padding=1, name="padding{}{}_branch2b".format(stage_char, block_char))(y)
        y = keras.layers.Conv2D(filters, kernel_size, use_bias=False, name="res{}{}_branch2b".format(stage_char, block_char), **parameters)(y)
        y = keras_resnet.layers.BatchNormalization(axis=axis, epsilon=1e-5, freeze=freeze_bn, name="bn{}{}_branch2b".format(stage_char, block_char))(y)
        y = keras.layers.Activation("relu", name="res{}{}_branch2b_relu".format(stage_char, block_char))(y)

        y = keras.layers.Conv2D(filters * 4, (1, 1), use_bias=False, name="res{}{}_branch2c".format(stage_char, block_char), **parameters)(y)
        y = keras_resnet.layers.BatchNormalization(axis=axis, epsilon=1e-5, freeze=freeze_bn, name="bn{}{}_branch2c".format(stage_char, block_char))(y)

        if block == 0:
            shortcut = keras.layers.Conv2D(filters * 4, (1, 1), strides=stride, use_bias=False, name="res{}{}_branch1".format(stage_char, block_char), **parameters)(x)
            shortcut = keras_resnet.layers.BatchNormalization(axis=axis, epsilon=1e-5, freeze=freeze_bn, name="bn{}{}_branch1".format(stage_char, block_char))(shortcut)
        else:
            shortcut = x

        y = keras.layers.Add(name="res{}{}".format(stage_char, block_char))([y, shortcut])
       
        print("y = ", y._keras_shape)
        y = keras.layers.Activation("relu", name="res{}{}_relu".format(stage_char, block_char))(y)

        return y

    return f

```
Signature: sparse_conv_lib.calc_block_params_res_block(in_size, bsize, ksize_list, strides, padding)  
Docstring:  
Calculates block parameters for a residual block.  

:param in_size:  [list]     List of 4 int. Size of the residual block input.  
:param bsize:    [list]     List of 4 int. Size of blocks, or downsample ratio, for each  
                            convolution layer in the residual block.  
:param ksize:    [list]     List of list of 4 int. Sparse convolution kernel size.  
:param strides:  [list]     List of 4 int. Sparse convolution stride size, for the first  
                            convolution in the residual block.  
                            Currently only supports when,  
                            1) (bsize[1] - ksize[0]) % strides[1] == 0 and,  
                            2) (bsize[2] - ksize[1]) % strides[2] == 0  
:param padding:  [string]   `VALID` or `SAME`, padding method for sparse convolution. 
```


In [31]:
def ResNet(inputs, blocks, block, include_top=True, classes=1000, freeze_bn=True, numerical_names=None, *args, **kwargs):
    if keras.backend.image_data_format() == "channels_last":
        axis = 3
    else:
        axis = 1
    if numerical_names is None:
        numerical_names = [True] * len(blocks)
    xsize = tf.cast(tf.shape(inputs), tf.float32)
    sparsity = 0.5
    mask = generate_mask(xsize, sparsity)
    block_params = sparse_conv_lib.calc_block_params([1, 224, 224, 3], [1, 5, 5, 1], [1, 1, 64, 64], [1,1,1,1], 'SAME')
    print(block_params)
    indices = sbnet_module.reduce_mask(
        mask, tf.constant(block_params.bcount, dtype=tf.int32),
        dynamic_bsize=block_params.bsize,
        dynamic_boffset=block_params.boffset,
        dynamic_bstride=block_params.bstrides,
        tol=0.5, # pooling threshold to consider a block as active
        avgpool=True) # max pooling by default
    x = keras.layers.ZeroPadding2D(padding=3, name="padding_conv1")(inputs)
    x = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name="conv1")(x)
    x = keras_resnet.layers.BatchNormalization(axis=axis, epsilon=1e-5, freeze=freeze_bn, name="bn_conv1")(x)
    x = keras.layers.Activation("relu", name="conv1_relu")(x)
    x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="pool1")(x)
    features = 64
    img_channels = 64
    outputs = []
    blockParams = { "dynamic_bsize": block_params.bsize, "dynamic_boffset": block_params.boffset, "dynamic_bstride": block_params.bstrides }
    for stage_id, iterations in enumerate(blocks):
        for block_id in range(iterations):
            blockStack = keras.layers.Lambda(lambda tmp: sbnet_module.sparse_gather(tmp, indices.bin_counts, indices.active_block_indices, transpose=False, **blockParams))(x)
            blockStack2 = keras.layers.Reshape((5, 5, img_channels))(blockStack)
            blockStack2._keras_shape = (1, 5, 5, img_channels)
            convBlocks = block(features, stage_id, block_id, numerical_name=(block_id > 0 and numerical_names[stage_id]), freeze_bn=freeze_bn)(blockStack2)
            if block_id == 0:
                if stage_id == 0:
                    x = K.concatenate([x, x, x, x], axis = 3)
                else:
                    x = K.concatenate([x, x], axis = 3)
                img_channels = features * 4;
            x = keras.layers.Lambda(lambda tmp: sbnet_module.sparse_scatter(
                tmp, indices.bin_counts, indices.active_block_indices,
                x, transpose=False, add=False, atomic=False, **blockParams)
                )(convBlocks)
            x = keras.layers.Reshape((56, 56, features * 4))(x)
            x._keras_shape = (1, 56, 56, features * 4)
        features *= 2
        outputs.append(x)
    if include_top:
        assert classes > 0
        x = keras.layers.GlobalAveragePooling2D(name="pool5")(x)
        x = keras.layers.Dense(classes, activation="softmax", name="fclast")(x)
        return keras.models.Model(inputs=inputs, outputs=x, *args, **kwargs)
    else:
        # Else output each stages features
        return keras.models.Model(inputs=inputs, outputs=outputs, *args, **kwargs)

In [32]:
#Building a more generalised ResNet50, with dynamic blocks
def ResNet50(inputs, blocks=None, include_top=True, classes=1000, b_size=1, *args, **kwargs):
    blocks = [2, 3, 4, 2]

    return ResNet(inputs, blocks, block=bottleneck_2d, include_top=include_top, classes=classes, *args, **kwargs)

In [33]:
inputs = keras.layers.Input(batch_shape=(1, 128, 128, 3)) #example

In [34]:
resnet = ResNet50(inputs, include_top=False, freeze_bn=True, b_size=b_size)

BlockParams(bsize=[5, 5], bsize_out=[5, 5], boffset=[0, 0], bcount=[45, 45], bstrides=[5, 5])
y =  (1, 5, 5, 256)
y =  (1, 5, 5, 256)
y =  (1, 5, 5, 512)
y =  (1, 5, 5, 512)
y =  (1, 5, 5, 512)
y =  (1, 5, 5, 1024)
y =  (1, 5, 5, 1024)
y =  (1, 5, 5, 1024)
y =  (1, 5, 5, 1024)
y =  (1, 5, 5, 2048)
y =  (1, 5, 5, 2048)


In [35]:
resnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (1, 128, 128, 3)     0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (1, 134, 134, 3)     0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (1, 64, 64, 64)      9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (1, 64, 64, 64)      256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu